In [17]:
import json
import requests
import time
from datetime import datetime
import pandas as pd
import csv
import os
import config

In [18]:
#Import the initial conditions 
key = config.api_key
lat = 51.838
lon = 0.1007
ini_timestamp = 1167609600    #Corresponding to 01/01/2007 00:00:00
final_timestamp = 1167696000   #Corresponding to 02/01/2007 00:00:00
timestamp = 0
units = 'SI' #Since the data in in International units (mts, °C...)

#We will initialise the dataframe to store the API response with the variables that will be required
names  = ['Station', 'latitude', 'longitude', 'time', 'cloudCover', 'humidity', 'temperature',
                        'temperatureMax', 'temperatureMaxTime', 'temperatureMin', 'temperatureMinTime',
                        'uvIndex', 'precipIntensitycurrent', 'precipTypecurrent', 'precipAccumulationcurrent',
                        'precipIntensityMax', 'precipIntensityMaxTime', 'precipType', 'precipAccumulation', 'units']
weather_df = pd.DataFrame(columns = names)

#Template for the API URL: Real time
dark_sky_url_rt= "https://api.darksky.net/forecast/"+ key +"/"+ str(lat)+","+str(lon)

#Template for the API URL : Historic with SI units, exclude: minutely, hourly, alerts, flags
dark_sky_url= "https://api.darksky.net/forecast/"+ key +"/"+ str(lat)+","+str(lon)+","+str(timestamp)+ "?exclude=minutely,hourly,flags&units=si"

In [19]:
#Difference in seconds between 8 hours, which will be the sample gap
diff = 28800

In [12]:
#Convert the datetime inf format '%d/%m/%Y %H:%M:%S' into a UNIX timestamp
s = '02/01/2007 00:00:00'
time = datetime.strptime(s, '%d/%m/%Y %H:%M:%S')
timestamp = datetime.timestamp(time)
print("timestamp =", timestamp)
print("Original datetime =", time)

timestamp = 1167696000.0
Original datetime = 2007-01-02 00:00:00


In [13]:
#Get the coordinates of all weather stations and store them into a dataframe
coordinates = pd.read_csv('Coordinates_Weather_stations.csv')

In [14]:
#Check that all weather stations are included 
#***We will only be doing some stations at a time, to prevent a system crash due to overload***
coordinates_test = coordinates.head(4)
print(len(coordinates_test))

4


In [15]:
print(coordinates_test)

                 Station        Lat       Lon
0              Cow Roast  51.783064 -0.613180
1            Offley Road  51.945884 -0.287471
2             Stapleford  51.835865 -0.100632
3  Bish Stortford Bypass  51.875576  0.152830


In [20]:
#Evaluate the number of runs for this loop:
a = 0
for lines in range(0,len(coordinates_test)):
    station = coordinates_test.iloc[lines,0]
    lat = coordinates_test.iloc[lines,1]
    lon = coordinates_test.iloc[lines,2]

    for timestamp in range(ini_timestamp,final_timestamp,diff):
        a = a+1
print(a)

12


In [ ]:
#For loop to request data from the URL and store it in a Dataframe
timestamp = ini_timestamp
#The loop will run for each one of the coordinates in the csv file:
for lines in range(0,len(coordinates_test)):
    station = coordinates_test.iloc[lines,0]
    lat = coordinates_test.iloc[lines,1]
    lon = coordinates_test.iloc[lines,2]

#The second for loop will run for the specified period (between initial and final time):    
    for timestamp in range(ini_timestamp,final_timestamp,diff):
      try:
        #Set the precipitation variables to 0
        precipIntensitycurrent = 0
        precipTypecurrent = '0'
        precipAccumulationcurrent = 0       
        precipIntensity = 0
        precipType = '0'
        precipAccumulation = 0
        precipIntensityMaxTime = 0
        
        #Request to Dark Sky API, ignoring the SSL cert validation, converting to JSON and extracting values:
        dark_sky_url= "https://api.darksky.net/forecast/"+ key +"/"+ str(lat)+","+str(lon)+","+str(timestamp)+ "?exclude=minutely,hourly,flags&units=si"
        response = requests.get(dark_sky_url, verify=False)
        data = response.json() 

        #Conditions for the specified time
        cloudCover = data['currently']['cloudCover']
        humidity = data['currently']['humidity']
        temperature = data['currently']['temperature']

        #Conditions for the day, overall
        temperatureMax = data['daily']['data'][0]['temperatureMax']
        temperatureMaxTime = data['daily']['data'][0]['temperatureHighTime']
        temperatureMin = data['daily']['data'][0]['temperatureLow']
        temperatureMinTime = data['daily']['data'][0]['temperatureLowTime']
        uvIndex = data['daily']['data'][0]['uvIndex']

        #Conditions for precipitation (daily and currently). 
        #***If there is no precipitation, data such as Type and accumulation will not appear in the dataset***

        if ('precipIntensity' in data['currently']) == True:
            precipIntensitycurrent = data['currently']['precipIntensity']
        if ('precipType' in data['currently']) == True:
            precipTypecurrent = data['currently']['precipType']
        if ('precipAccumulation' in data['currently']) == True:
            precipAccumulationcurrent = data['currently']['precipAccumulation']
        if ('precipIntensityMax' in data['daily']['data'][0]) == True:
            precipIntensityMax = data['daily']['data'][0]['precipIntensityMax']
        if ('precipType' in data['daily']['data'][0]) == True:
            precipType = data['daily']['data'][0]['precipType']
        if ('precipAccumulation' in data['daily']['data'][0]) == True:
            precipAccumulation = data['daily']['data'][0]['precipAccumulation']
        if ('precipIntensityMaxTime' in data['daily']['data'][0]) == True:
            precipIntensityMaxTime = data['daily']['data'][0]['precipIntensityMaxTime']

        #We store the data into a DataFrame and append the data to the weather DataFramce
        variable  = pd.DataFrame([[station, lat, lon, timestamp, cloudCover, humidity, temperature,
                                temperatureMax, temperatureMaxTime, temperatureMin, temperatureMinTime,
                                uvIndex, precipIntensitycurrent, precipTypecurrent, precipAccumulationcurrent,
                                precipIntensityMax, precipIntensityMaxTime, precipType, 
                                precipAccumulation, units]], columns = names)

        weather_df = weather_df.append(variable, ignore_index=True)
        
        
        
      except:
        print("This was an exception")
        precipIntensitycurrent = 'NA'
        precipTypecurrent = 'NA'
        precipAccumulationcurrent = 'NA'       
        precipIntensity = 'NA'
        precipType = 'NA'
        precipAccumulation = 'NA'
        
        #We store the data into a DataFrame and append the data to the weather DataFramce
        variable  = pd.DataFrame([[station, lat, lon, timestamp, cloudCover, humidity, temperature,
                                temperatureMax, temperatureMaxTime, temperatureMin, temperatureMinTime,
                                uvIndex, precipIntensitycurrent, precipTypecurrent, precipAccumulationcurrent,
                                precipIntensityMax, precipIntensityMaxTime, precipType, 
                                precipAccumulation, units]], columns = names)

        weather_df = weather_df.append(variable, ignore_index=True)
       

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

...

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

This was an exception


C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\J FernandezGomez\AppData\Local\Continuum\anaconda3\lib\site-packages\urllib3\connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\User

In [ ]:
weather_df.head(10)

In [ ]:
#Save the Dataframe to a CSV File and save it to the specified path
weather_df.to_csv(r'C:\\Users\\J FernandezGomez\\Jupyter Notebooks\\weather_consolidate_3.csv', index=False, header = True)